# The future home of an awesome ML project.
Please keep things commented and use lots of Markdown cells to separate sections or steps of the process, or to provide additional explanation.

In [1]:
#!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
#import <package_name>